# Create a View of Source Table `[Uploadfile].[CurrentSemesterCourseOfferings]`


In [21]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create View `[Uploadfile].[vw_CurrentSemesterCourseOfferings]` from source table `[Uploadfile].[CurrentSemesterCourseOfferings]`. 
--                ChatGPT 4o assisted >
-- =============================================

USE QueensClassSchedule
GO

CREATE VIEW [Uploadfile].[vw_CurrentSemesterCourseOfferings] AS
SELECT
    [Semester],
    [Sec],
    [Code],
    [Course (hr, crd)] AS OriginalCourseColumn,
    -- Extract CourseName (text before the first '(')
    LTRIM(RTRIM(LEFT([Course (hr, crd)], CHARINDEX('(', [Course (hr, crd)]) - 1))) AS CourseName,
    -- Extract hr (number between '(' and ',')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX('(', [Course (hr, crd)]) + 1,
        CHARINDEX(',', [Course (hr, crd)]) - CHARINDEX('(', [Course (hr, crd)]) - 1))) AS hr,
    -- Extract crd (number between ',' and ')')
    LTRIM(RTRIM(SUBSTRING([Course (hr, crd)],
        CHARINDEX(',', [Course (hr, crd)]) + 1,
        CHARINDEX(')', [Course (hr, crd)]) - CHARINDEX(',', [Course (hr, crd)]) - 1))) AS crd,
    [Description],
    [Day],
    [Time],
    [Instructor],
    [Location],
    [Enrolled],
    [Limit],
    [Mode of Instruction]
FROM
    [Uploadfile].[CurrentSemesterCourseOfferings];


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.014

# Create the required schemas and User-defined Data Types

### Schemas being created:
- `[Udt]`
- `[DbSecurity]`
- `[Course]`
- `[Location]`
- `[Department]`
- `[Process]`
- `[Project3]`

In [22]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski, Jascharan Singh, Sarmad Ali, Augusta Na, Carlos Vega >
-- Create date: < 12/3/2024 >
-- Description:	< Create the required schemas and UDT's >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [Udt]
GO

CREATE  TYPE    [Udt].[SurrogateKeyInt]     FROM    [int]               NULL
GO
CREATE  TYPE    [Udt].[DateAdded]           FROM    Datetime2(7)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassTime]           FROM    NCHAR(5)        NOT NULL
GO
CREATE  TYPE    [Udt].[IndividualProject]   FROM    NVARCHAR(60)    NOT NULL
GO
CREATE  TYPE    [Udt].[LastName]            FROM    NVARCHAR(35)    NOT NULL
GO
CREATE  TYPE    [Udt].[FirstName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[GroupName]           FROM    NVARCHAR(20)    NOT NULL
GO
CREATE  TYPE    [Udt].[String100]           FROM    NVARCHAR(100)   NOT NULL
GO
CREATE  TYPE    [Udt].[Rows]                FROM    [int]           NOT NULL
GO
CREATE  TYPE    [Udt].[String4]             FROM    NVARCHAR(4)         NULL
GO
CREATE  TYPE    [Udt].[String8]             FROM    NVARCHAR(8)         NULL
GO
CREATE  TYPE    [Udt].[String20]            FROM    NVARCHAR(20)        NULL
GO
CREATE  TYPE    [Udt].[String30]            FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[ClassLength]         FROM    NVARCHAR(30)    NOT NULL
GO
CREATE  TYPE    [Udt].[FloatField]          FROM    FLOAT           NOT NULL
GO

-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the required schemas >
-- =============================================

USE QueensClassSchedule
GO

CREATE  SCHEMA  [DbSecurity]
GO
CREATE  SCHEMA  [Course]
GO
CREATE  SCHEMA  [Location]
GO
CREATE  SCHEMA  [Department]
GO
CREATE  SCHEMA  [Process]
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.070

# Create `[DbSecurity].[UserAuthorization]` Table

### Create and populate the table storing the group members:

In [23]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the `[DbSecurity].[UserAuthorization]` table >
-- =============================================

USE QueensClassSchedule
GO

CREATE  TABLE   [DbSecurity].[UserAuthorization]    (
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [ClassTime]             [Udt].[ClassTime]                           NULL    DEFAULT ('10:45'),
    [IndividualProject]     [Udt].[IndividualProject]                   NULL    DEFAULT ('Project 3'),
    [GroupMemberLastName]   [Udt].[LastName]                        NOT NULL,
    [GroupMemberFirstName]  [Udt].[FirstName]                       NOT NULL,
    [GroupName]             [Udt].[GroupName]                       NOT NULL    DEFAULT ('Group 2'),
    [DateAdded]             [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME())
);
GO

INSERT  INTO    [DbSecurity].[UserAuthorization]    (GroupMemberFirstName,   GroupMemberLastName)
VALUES
    ('Sarmad',      'Ali'), 
    ('Daniel',      'Gargiullo'), 
    ('Augusta',     'Na'), 
    ('Jascharan',   'Singh'),
    ('Carlos',      'Vega Lemus'), 
    ('Oksana',      'Weigand-Suminski');

Commands completed successfully.

Commands completed successfully.

(6 rows affected)

Total execution time: 00:00:00.012

# Create Required Tables
- `[Process]`
    - `.[WorkFlowSteps]`
- `[Course]`
    - `.[Course]`
    - `.[Class]`
    - `.[ModeOfInstruction]`
- `[Location]`
    - `.[BuildingLocation]`
    - `.[RoomLocation]`
- `[Department]`
    - `.[Department]`
    - `.[Instructor]`

In [24]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/4/2024 >
-- Description:	< Create the other required tables >
-- =============================================

USE QueensClassSchedule
GO

/* [Process] schema */

CREATE  TABLE   [Process].[WorkFlowSteps]   (
    [WorkFlowStepsKey]              [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [WorkFlowStepsDescription]      [Udt].[String100]                       NOT NULL,
    [WorkFlowStepsTableRowCount]    [Udt].[Rows]                                NULL    DEFAULT ((0)),
    [StartingDateTime]              [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [EndDateTime]                   [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [ClassTime]                     [Udt].[ClassTime]                           NULL    DEFAULT ('10:45'),

    [UserAuthorizationKey]          [Udt].[SurrogateKeyInt]                 NOT NULL
)
GO

/* [Course] schema */

CREATE  TABLE   [Course].[Course]   (
	[CourseID]              [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Section]               [Udt].[String4]                             NULL,
	[Course]                [Udt].[String30]                            NULL,
    [Semester]              [Udt].[String20]                            NULL,
	[Description]           [Udt].[String100]                           NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

CREATE  TABLE   [Course].[Class]    (
    [ClassID]               [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Enrollment]            [Udt].[SurrogateKeyInt]                     NULL,
    [Limit]                 [Udt].[SurrogateKeyInt]                     NULL,
    [Section]               [Udt].[String30]                            NULL,
    [Department]            [Udt].[String30]                            NULL,
    [Hours]                 [Udt].[FloatField]                          NULL,
    [Credits]               [Udt].[SurrogateKeyInt]                     NULL,
    [Days]                  [Udt].[String30]                            NULL,
    [ClassStart]            [Udt].[String20]                            NULL,
    [ClassEnd]              [Udt].[String20]                            NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME()),
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL    DEFAULT (SYSDATETIME())
)
GO

CREATE  TABLE   [Course].[ModeOfInstruction]    (
	[ModeID]                [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
	[ModeName]              [Udt].[String100]                       NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
);

-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski, Carlos Vega >
-- Create date: < 12/5/2024 >
-- Description:	< Create the required tables in the `[Location]` schema >
-- =============================================

/* [Location] schema */

CREATE  TABLE   [Location].[BuildingLocation]   (
    [BuildingLocationID]    [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL 
        PRIMARY KEY CLUSTERED,
    [CourseCode]            [Udt].[String30]                        NOT NULL,
    [Description]           [Udt].[String100]                           NULL,
    [Building]              [Udt].[String30]                        NOT NULL,
    [Day]                   [Udt].[String30]                            NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL,
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

-- ======================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/5/2024 >
-- Description:	< Create the required tables in the `[Location]` schema >
-- ======================================================================

CREATE  TABLE   [Location].[RoomLocation]   (
	[RoomLocationID]        [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Room]                  [Udt].[String30]                        NOT NULL,
    [Time]                  [Udt].[String20]                        NOT NULL,
    [Day]                   [Udt].[String20]                        NOT NULL,
	[BuildingLocationID]    [Udt].[SurrogateKeyInt]                 NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

/* [Department] schema */

CREATE  TABLE   [Department].[Department]   (
    [DepartmentID]          [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [Department]            [Udt].[String8]                         NOT NULL,
    [InstructorID]          [Udt].[SurrogateKeyInt]                 NOT NULL,
    [FirstName]             [Udt].[String30]                        NOT NULL,
    [LastName]              [Udt].[String30]                        NOT NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL,
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

CREATE  TABLE   [Department].[Instructor]   (
    [InstructorID]          [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [DepartmentID]          [Udt].[SurrogateKeyInt]                 NOT NULL,
    [FirstName]             [Udt].[String30]                        NOT NULL,
    [LastName]              [Udt].[String30]                        NOT NULL,
    -- Required fields:
	[UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
	[DateAdded]             [Udt].[DateAdded]                           NULL,
	[DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.077

# Create Workflow Procedures

- `[Process]`
    - `[usp_TrackWorkFlow]`
    - `[usp_ShowWorkFlowSteps]`

In [25]:
-- =============================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/5/2024 >
-- Description:	< Create `[Process].[usp_TrackWorkFlow]` and `[Process].[usp_ShowWorkFlowSteps]` procedures >
-- =============================================

USE QueensClassSchedule
GO

CREATE PROCEDURE [Process].[usp_TrackWorkFlow]
    @UserAuthorizationKey INT,
    @WorkFlowStepsTableRowCount INT,
    @WorkFlowStepsDescription NVARCHAR(100)
AS
BEGIN
    INSERT INTO [Process].[WorkFlowSteps]
        (UserAuthorizationKey, WorkFlowStepsTableRowCount, WorkFlowStepsDescription)
    VALUES(@UserAuthorizationKey, @WorkFlowStepsTableRowCount, @WorkFlowStepsDescription);
END;
GO

CREATE PROCEDURE [Process].[usp_ShowWorkFlowSteps] 
AS
BEGIN
	SELECT *
	FROM [Process].[WorkFlowSteps];
END;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.011

# Create and Execute Stored Procedures

- `[Course]`
    - `.[usp_LoadClassTable]`
    - `.[usp_LoadModeOfInstructionTable]`
    - `.[usp_LoadCourseTable]`
- `[Location]`
    - `.[usp_LoadBuildingLocationTable]`
    - `.[usp_LoadRoomLocationTable]`
- `[Department]`
    - `.[Department]`

In [26]:
-- ================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/6/2024 >
-- Description:	< Create the `[Process].[usp_TrackWorkFlowSteps]` procedure >
-- ================================================================

USE QueensClassSchedule
GO

CREATE PROCEDURE [Course].[usp_LoadCourseTable]
    @UserAuthorizationKey INT
AS
BEGIN
    INSERT INTO Course.Course
    (
        Section, 
        Course, 
        [Semester], 
        [Description], 
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT DISTINCT
        [Sec] AS Section, 
        SUBSTRING([Course (hr, crd)], 1, CHARINDEX('(', [Course (hr, crd)]) - 1) AS Course,
        [Semester],
        [Description],
        @UserAuthorizationKey,
        SYSDATETIME() AS DateAdded, 
        SYSDATETIME() AS DateOfLastUpdate
    FROM Uploadfile.CurrentSemesterCourseOfferings;
    EXEC [Process].[usp_TrackWorkFlow] 
        @UserAuthorizationKey = @UserAuthorizationKey, 
        @WorkFlowStepsDescription = 'Loading data into Course table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

CREATE PROCEDURE [Course].[usp_LoadClassTable]
    @UserAuthorizationKey INT
AS
BEGIN
    INSERT INTO [Course].[Class] (
        Section,
        Department,
        Hours,
        Credits,
        Days,
        ClassStart,
        ClassEnd,
        Enrollment,
        [Limit],
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT
        [Sec] AS Section,
        LEFT([Course (hr, crd)], CHARINDEX(' ', [Course (hr, crd)]) - 1) AS Department,
        TRY_CAST(
            TRIM(
                SUBSTRING(
                    [Course (hr, crd)],
                    CHARINDEX('(', [Course (hr, crd)]) + 1,
                    CHARINDEX(',', [Course (hr, crd)]) - CHARINDEX('(', [Course (hr, crd)]) - 1
                )
            ) AS FLOAT
        ) AS Hours,
        TRY_CAST(
            TRIM(
                SUBSTRING(
                    [Course (hr, crd)],
                    CHARINDEX(',', [Course (hr, crd)]) + 1,
                    CHARINDEX(')', [Course (hr, crd)]) - CHARINDEX(',', [Course (hr, crd)]) - 1
                )
            ) AS INT
        ) AS Credits,
        [Day] AS Days,
        TRIM(LEFT([Time], CHARINDEX('-', [Time]) - 1)) AS ClassStart,
        TRIM(SUBSTRING([Time], CHARINDEX('-', [Time]) + 1, LEN([Time]))) AS ClassEnd,
        TRY_CAST([Enrolled] AS INT) AS Enrollment,
        TRY_CAST([Limit] AS INT) AS [Limit],
        @UserAuthorizationKey,
        SYSDATETIME() AS DateAdded,
        SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings] c
    WHERE
        [Course (hr, crd)] IS NOT NULL
        AND CHARINDEX(' ', [Course (hr, crd)]) > 0
        AND CHARINDEX('-', [Time]) > 0;
    EXEC [Process].[usp_TrackWorkFlow] 
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription = 'Loading data into Class table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

CREATE PROCEDURE [Course].[usp_LoadModeOfInstructionTable]
    @UserAuthorizationKey INT
AS
BEGIN
    INSERT INTO [Course].[ModeOfInstruction]
    (
        ModeName,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT [Mode of Instruction] AS ModeName, @UserAuthorizationKey, SYSDATETIME() AS DateAdded, SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings]
    --  Insert the user into the Process.WorkFlowTable
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription =  'Loading data into MOI table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

CREATE PROCEDURE [Location].[usp_LoadBuildingLocationTable]
    @UserAuthorizationKey INT
AS
BEGIN
    -- Insert data into the BuildingLocation table
    INSERT INTO [Location].[BuildingLocation]
    (
        CourseCode,
        Description,
        Building,
        Day,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT 
        [Code] AS CourseCode,
        [Description],
        TRIM(LEFT([Location], CHARINDEX(' ', [Location]) - 1)) AS Building,
        [Day],
        @UserAuthorizationKey AS UserAuthorizationKey,
        SYSDATETIME() AS DateAdded,
        SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings]
    WHERE 
        [Location] IS NOT NULL
        AND CHARINDEX(' ', [Location]) > 0;
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription = 'Loading data into BuildingLocation table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

CREATE PROCEDURE [Location].[usp_LoadRoomLocationTable]
    @UserAuthorizationKey INT
AS
BEGIN
    -- Insert data into the RoomLocation table
    INSERT INTO [Location].[RoomLocation]
    (
        Room,
        Time,
        Day,
        BuildingLocationID,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT 
        TRIM(SUBSTRING(c.[Location], CHARINDEX(' ', c.[Location]) + 1, LEN(c.[Location]))) AS Room,
        c.[Time],
        c.[Day], -- Explicitly use table alias
        bl.BuildingLocationID,
        @UserAuthorizationKey AS UserAuthorizationKey,
        SYSDATETIME() AS DateAdded,
        SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings] c
    INNER JOIN [Location].[BuildingLocation] bl
        ON bl.CourseCode = c.[Code]
    WHERE
        c.[Location] IS NOT NULL
        AND CHARINDEX(' ', c.[Location]) > 0;
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription = 'Loading data into RoomLocation table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

-- INSTRUCTOR PROCEDURE
CREATE  PROCEDURE [Department].[usp_LoadInstructorTable]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;
    -- Insert data into Department.Instructor table
    INSERT INTO [Department].[Instructor]
    (
        LastName,
        FirstName,
        DepartmentID, -- Add this column
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT DISTINCT
        SUBSTRING([Instructor], 1, CHARINDEX(',', [Instructor]) - 1) AS LastName,
        LTRIM(SUBSTRING([Instructor], CHARINDEX(',', [Instructor]) + 1, LEN([Instructor]) - CHARINDEX(',', [Instructor]))) AS FirstName,
        1 AS DepartmentID, -- Replace with an appropriate value or logic
        @UserAuthorizationKey,
        SYSDATETIME() AS DateAdded,
        SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings]
    WHERE [Instructor] IS NOT NULL
          AND CHARINDEX(',', [Instructor]) > 0;
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey, 
        @WorkFlowStepsDescription = 'Loading data into Instructor table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

-- DEPARTMENT PROCEDURE
CREATE PROCEDURE [Department].[usp_LoadDepartmentTable]
    @UserAuthorizationKey INT
AS
BEGIN

    INSERT INTO [Department].[Department]
    (
        Department,   
        InstructorID,
        FirstName,
        LastName,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT DISTINCT
        -- Extract department code (e.g., ACCT from ACCT 100)
        LEFT([Course (hr, crd)], CHARINDEX(' ', [Course (hr, crd)]) - 1) AS Department,

        -- InstructorID from the Instructor table
        inst.InstructorID,

        -- Extract LastName from Instructor column
        SUBSTRING(cso.[Instructor], 1, CHARINDEX(',', cso.[Instructor]) - 1) AS LastName,

        -- Extract FirstName from Instructor column
        LTRIM(SUBSTRING(cso.[Instructor], CHARINDEX(',', cso.[Instructor]) + 1, LEN(cso.[Instructor]) - CHARINDEX(',', cso.[Instructor]))) AS FirstName,

        @UserAuthorizationKey,
        SYSDATETIME() AS DateAdded,
        SYSDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings] cso
    INNER JOIN [Department].[Instructor] inst 
        ON inst.LastName = SUBSTRING(cso.[Instructor], 1, CHARINDEX(',', cso.[Instructor]) - 1)
        AND inst.FirstName = LTRIM(SUBSTRING(cso.[Instructor], CHARINDEX(',', cso.[Instructor]) + 1, LEN(cso.[Instructor]) - CHARINDEX(',', cso.[Instructor])))
    WHERE [Course (hr, crd)] IS NOT NULL
          AND CHARINDEX(' ', [Course (hr, crd)]) > 0
          AND cso.[Instructor] IS NOT NULL
          AND CHARINDEX(',', cso.[Instructor]) > 0;

    -- Log the workflow step
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey, 
        @WorkflowStepsDescription = 'Loading data into Department table',
        @WorkflowStepsTableRowCount = @@ROWCOUNT;
END;
GO

EXEC    [Course].[usp_LoadCourseTable]              @UserAuthorizationKey = 6
GO
EXEC    [Course].[usp_LoadClassTable]               @UserAuthorizationKey = 6
GO
EXEC    [Course].[usp_LoadModeOfInstructionTable]   @UserAuthorizationKey = 2
GO
EXEC    [Location].[usp_LoadBuildingLocationTable]  @UserAuthorizationKey = 2
GO
EXEC    [Location].[usp_LoadRoomLocationTable]      @UserAuthorizationKey = 2
GO
EXEC    [Department].[usp_LoadInstructorTable]      @UserAuthorizationKey = 4
GO
EXEC    [Department].[usp_LoadDepartmentTable]      @UserAuthorizationKey = 6
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

(1 row affected)

Commands completed successfully.

(1755 rows affected)

(1 row affected)

Total execution time: 00:00:00.240

# Alter existing tables to have foreign key relationships

- Add Foreign Keys referencing `[DbSecurity].[UserAuthorization]`
- Add Foreign Key for Department-Instructor relationship`
    - We assume `Instructor(DepartmentID)` references `Department(DepartmentID)`
- Add Foreign Key for `[Location]` tables relationship
    - `RoomLocation(BuildingLocationID)` references `BuildingLocation(BuildingLocationID)`

In [27]:
-- ================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/6/2024 >
-- Description:	< Create foreign key relationships >
-- ================================================================

USE [QueensClassSchedule];
GO

ALTER TABLE [Course].[Class]
ADD CONSTRAINT FK_Class_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Course].[Course]
ADD CONSTRAINT FK_Course_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Course].[ModeOfInstruction]
ADD CONSTRAINT FK_ModeOfInstruction_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Department].[Department]
ADD CONSTRAINT FK_Department_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Department].[Instructor]
ADD CONSTRAINT FK_Instructor_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Location].[BuildingLocation]
ADD CONSTRAINT FK_BuildingLocation_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Location].[RoomLocation]
ADD CONSTRAINT FK_RoomLocation_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Process].[WorkFlowSteps]
ADD CONSTRAINT FK_WorkFlowSteps_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);

ALTER TABLE [Department].[Instructor]
ADD CONSTRAINT FK_Instructor_Department FOREIGN KEY (DepartmentID)
REFERENCES [Department].[Department](DepartmentID);

ALTER TABLE [Location].[RoomLocation]
ADD CONSTRAINT FK_RoomLocation_BuildingLocation FOREIGN KEY (BuildingLocationID)
REFERENCES [Location].[BuildingLocation](BuildingLocationID);

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.079

# Create Bridge Table `[Course].[CourseInstructor]`

In [28]:
-- ================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/7/2024 >
-- Description:	< Create the `[Course].[CourseInstructor]` table, which is a bridge between `[Course].[Course]` and `[Department].[Instructor]` >
-- ================================================================

USE [QueensClassSchedule];
GO

CREATE TABLE [Course].[CourseInstructor]   (
    [CourseInstructorID]          [Udt].[SurrogateKeyInt] IDENTITY(1,1)   NOT NULL
        PRIMARY KEY CLUSTERED,
    [CourseID]              [Udt].[SurrogateKeyInt]                 NOT NULL,
    [InstructorID]          [Udt].[SurrogateKeyInt]                 NOT NULL,
    -- Required fields:
    [UserAuthorizationKey]  [Udt].[SurrogateKeyInt]                     NULL,
    [DateAdded]             [Udt].[DateAdded]                           NULL,
    [DateOfLastUpdate]      [Udt].[DateAdded]                           NULL
)
GO

-- Add foreign keys to enforce referential integrity
ALTER TABLE [Course].[CourseInstructor]
ADD CONSTRAINT FK_CourseInstructor_Course FOREIGN KEY (CourseID)
    REFERENCES [Course].[Course](CourseID);

ALTER TABLE [Course].[CourseInstructor]
ADD CONSTRAINT FK_CourseInstructor_Instructor FOREIGN KEY (InstructorID)
    REFERENCES [Department].[Instructor](InstructorID);

ALTER TABLE [Course].[CourseInstructor]
ADD CONSTRAINT FK_CourseInstructor_UserAuthorization FOREIGN KEY (UserAuthorizationKey)
    REFERENCES [DbSecurity].[UserAuthorization](UserAuthorizationKey);


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

# Create and execute Stored Procedure for loading `[Course].[CourseInstructor]`

In [29]:
-- ================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski >
-- Create date: < 12/7/2024 >
-- Description:	< Create the `[Process].[usp_TrackWorkFlowSteps]` procedure >
-- ================================================================

USE [QueensClassSchedule];
GO

CREATE PROCEDURE [Course].[usp_LoadCourseInstructorTable]
    @UserAuthorizationKey INT
AS
BEGIN

    INSERT INTO [Course].[CourseInstructor]
    (
        CourseID,
        InstructorID,
        UserAuthorizationKey,
        DateAdded,
        DateOfLastUpdate
    )
    SELECT DISTINCT 
        crs.CourseID,
        inst.InstructorID,
        @UserAuthorizationKey,
        SYSUTCDATETIME() AS DateAdded,
        SYSUTCDATETIME() AS DateOfLastUpdate
    FROM [Uploadfile].[CurrentSemesterCourseOfferings] cso
    INNER JOIN [Course].[Course] crs 
        ON crs.[Course] = LTRIM(RTRIM(SUBSTRING(cso.[Course (hr, crd)], 1, CHARINDEX('(', cso.[Course (hr, crd)]) - 1)))
        AND crs.[Section] = cso.[Sec]
        AND crs.[Semester] = cso.[Semester]
    INNER JOIN [Department].[Instructor] inst 
        ON inst.LastName = SUBSTRING(cso.[Instructor], 1, CHARINDEX(',', cso.[Instructor]) - 1)
        AND inst.FirstName = LTRIM(SUBSTRING(cso.[Instructor], CHARINDEX(',', cso.[Instructor]) + 1, LEN(cso.[Instructor]) - CHARINDEX(',', cso.[Instructor])));
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription = 'Loading data into CourseInstructor bridge table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

EXEC    [Course].[usp_LoadCourseInstructorTable]      @UserAuthorizationKey = 2
GO

Commands completed successfully.

Commands completed successfully.

(4424 rows affected)

(1 row affected)

Total execution time: 00:00:00.072

# Create Bridge Table `[Course].[ClassRoom]`

### Load the `[Course].[ClassRoom]` table by creating and executing the `[Course].[usp_LoadClassRoomTable]` stored procedure

In [30]:
-- ================================================================
-- Author:		< Daniel Gargiullo, Oksana Weigand-Suminski , Carlos Vega>
-- Create date: < 12/7/2024 >
-- Description:	< Create the `[Course].[ClassRoom]` table, which is a bridge between `[Course].[Class]` and `[Location].[RoomLocation]` >
-- ================================================================

USE [QueensClassSchedule];
GO

-- Create the ClassRoom bridge table.
CREATE TABLE [Course].[ClassRoom](
    [ClassRoomID] [Udt].[SurrogateKeyInt] IDENTITY(1,1) NOT NULL,
    [ClassID] [Udt].[SurrogateKeyInt] NOT NULL,
    [RoomLocationID] [Udt].[SurrogateKeyInt] NOT NULL,
    [UserAuthorizationKey] [Udt].[SurrogateKeyInt] NULL,
    [DateAdded] [Udt].[DateAdded] NULL,
    [DateOfLastUpdate] [Udt].[DateAdded] NULL,
    CONSTRAINT [PK_ClassRoom] PRIMARY KEY CLUSTERED ([ClassRoomID] ASC)
        WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, 
              IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, 
              ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
) ON [PRIMARY];
GO

-- Set default values for DateAdded and DateOfLastUpdate
ALTER TABLE [Course].[ClassRoom]
    ADD CONSTRAINT [DF_ClassRoom_DateAdded] 
    DEFAULT (sysdatetime()) FOR [DateAdded];
GO

ALTER TABLE [Course].[ClassRoom]
    ADD CONSTRAINT [DF_ClassRoom_DateOfLastUpdate] 
    DEFAULT (sysdatetime()) FOR [DateOfLastUpdate];
GO

-- Add foreign key constraints
ALTER TABLE [Course].[ClassRoom] WITH CHECK 
ADD CONSTRAINT [FK_ClassRoom_Class] 
FOREIGN KEY([ClassID])
REFERENCES [Course].[Class]([ClassID]);
GO
ALTER TABLE [Course].[ClassRoom] CHECK CONSTRAINT [FK_ClassRoom_Class];
GO

ALTER TABLE [Course].[ClassRoom] WITH CHECK 
ADD CONSTRAINT [FK_ClassRoom_RoomLocation] 
FOREIGN KEY([RoomLocationID])
REFERENCES [Location].[RoomLocation]([RoomLocationID]);
GO
ALTER TABLE [Course].[ClassRoom] CHECK CONSTRAINT [FK_ClassRoom_RoomLocation];
GO

-- Optional: Tie UserAuthorizationKey to UserAuthorization table for auditing consistency
ALTER TABLE [Course].[ClassRoom] WITH CHECK 
ADD CONSTRAINT [FK_ClassRoom_UserAuthorization]
FOREIGN KEY([UserAuthorizationKey])
REFERENCES [DbSecurity].[UserAuthorization]([UserAuthorizationKey]);
GO
ALTER TABLE [Course].[ClassRoom] CHECK CONSTRAINT [FK_ClassRoom_UserAuthorization];
GO


-- Create a stored procedure to populate the ClassRoom table
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO

CREATE PROCEDURE [Course].[usp_LoadClassRoomTable]
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    -- Insert the bridging records by matching Classes and RoomLocations based on Day and Time
    -- Class time = ClassStart + '-' + ClassEnd
    INSERT INTO [Course].[ClassRoom]
    (
        [ClassID],
        [RoomLocationID],
        [UserAuthorizationKey],
        [DateAdded],
        [DateOfLastUpdate]
    )
    SELECT
        cls.ClassID,
        rl.RoomLocationID,
        @UserAuthorizationKey,
        SYSDATETIME(),
        SYSDATETIME()
    FROM [Course].[Class] cls
    INNER JOIN [Location].[RoomLocation] rl
        ON cls.Days = rl.Day
        AND (cls.ClassStart + '-' + cls.ClassEnd) = rl.Time;

    -- Log the workflow step
    EXEC [Process].[usp_TrackWorkFlow]
        @UserAuthorizationKey = @UserAuthorizationKey,
        @WorkFlowStepsDescription = 'Loading data into ClassRoom bridge table',
        @WorkFlowStepsTableRowCount = @@ROWCOUNT;
END;
GO

EXEC    [Course].[usp_LoadClassRoomTable]      @UserAuthorizationKey = 5
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

...

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.041